In [1]:
import numpy as np
import open3d as o3d
import pathlib
import re
from ipywidgets import interact
from lib.datasets.kitti_utils import Calibration
import cv2

%load_ext autoreload
%autoreload 2

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [40]:
from pathlib import Path
from tools.sample_util import SampleDatabase
from tools.dataset_util import Dataset
test_dir = Path("/mnt/e/DataSet/kitti/kitti_img_database/test")
np.random.seed(0)

idx = 6

database = SampleDatabase("/mnt/e/DataSet/kitti/kitti_img_database/")
dataset = Dataset("train", r"/mnt/e/DataSet/kitti")

calib_ = dataset.get_calib(idx)
image, depth = dataset.get_image_with_depth(idx, use_penet=True)
ground, non_ground = dataset.get_lidar_with_ground(idx, fov=True)
plane_ = dataset.get_plane(idx)
samples = database.get_samples(ground, non_ground, calib_, plane_)
image_, depth_, flag = database.add_samples_to_scene(samples, image, depth)

#### Patchwork 地面分割演示

In [41]:
from tools.visualize_util import show_o3d

rgb_ground = np.zeros_like(ground) + np.array([[255, 0, 0]])
rgb_non_ground = np.zeros_like(non_ground) + np.array([[0, 255, 0]])

lidar = np.concatenate([ground, non_ground], axis=0)
rgb = np.concatenate([rgb_ground, rgb_non_ground], axis=0)

show_o3d(lidar, rgb)

#### 样本间重叠筛除演示

In [42]:
samples, bbox3d = database.sample_with_fixed_number(calib_, plane_)
bbox3d_, flag1 = database.sample_put_on_plane(bbox3d, ground)

from tools.box_util import rect2lidar, boxes_bev_iou_cpu
# api 要求 bbox3d 为 lidar 坐标系
bbox3d_in_lidar = rect2lidar(bbox3d_[flag1], calib_)

# 判断样本间是否有重叠，第二次筛除
iou = boxes_bev_iou_cpu(bbox3d_in_lidar, bbox3d_in_lidar)
iou

array([[1.0000006 , 0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.9999984 , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 1.0000004 , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.99999887, 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.9999998 ]],
      dtype=float32)

#### 障碍物重叠筛除演示

In [43]:
show_o3d(non_ground, bbox3d=bbox3d_[flag1])

#### 生成场景演示

In [16]:
from tools.visualize_util import show_o3d
from tools.sample_util import to3d

cord, rgb = to3d(image_, depth_, calib_)
show_o3d(cord, rgb, bbox3d=np.array([sample.bbox3d_ for sample in samples]))

#### ry 修正演示

In [35]:
xyz_ = np.array([0, 0, 25])
xyz_ = np.repeat(xyz_.reshape(1, -1), 10, axis=0)

xyz_[:, 0] += np.arange(-20, 20, 4)
xyz_

array([[-20,   0,  25],
       [-16,   0,  25],
       [-12,   0,  25],
       [ -8,   0,  25],
       [ -4,   0,  25],
       [  0,   0,  25],
       [  4,   0,  25],
       [  8,   0,  25],
       [ 12,   0,  25],
       [ 16,   0,  25]])

In [39]:
from tools.sample_util import to3d, Sample
samples, bbox3d = database.sample_with_fixed_idx(xyz_, calib_, 0)
samples = [Sample(samples[i], bbox3d[i], calib_, database) for i in range(len(samples))]
cords, rgbs = [], []
for i in range(len(samples)):
    cord, rgb = to3d(samples[i].image_, samples[i].depth_, calib_, samples[i].bbox2d_)
    cords.append(cord)
    rgbs.append(rgb)
    
cord = np.concatenate([*cords], axis=0)
rgb = np.concatenate([*rgbs], axis=0)

show_o3d(cord, rgb, bbox3d=np.array([sample.bbox3d_ for sample in samples]))